In [ ]:
import numpy as np

import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from argparse import Namespace
import seaborn as sns 
from sklearn import preprocessing
from warnings import filterwarnings
filterwarnings('ignore')

sns.set(rc={'figure.figsize':(10,8)})



In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE

In [ ]:
args = Namespace(
    target="FiyatAraligi",
    data_file="tablet.csv",
    cv = 10
)

İşe kütüphanelerimizi eklemekle başlıyoruz.

In [ ]:
df = pd.read_csv('../input/tabletpc-priceclassification/tablet.csv')

veri setimizden bilgileri yazdırma işlemini df.head() komuduyla yapabiliriz

veri çerçevemize genel olarak bakmaya başlayabiliriz

In [ ]:
df.head()


In [ ]:
df.info()


In [ ]:
 df.describe().T #sayısal verileri görmemizi sağlar

In [ ]:
df.tail() #veri çerçevesinin son 5 gözlemini görüntüledik.

toplamda görüldüğü üzere 2000 akıllı tabletle ilgili bir data setimiz bulunmaktadır


In [ ]:
df.sample(10)
# rastgele 10 tane gözlemi görüntüler

In [ ]:
df.shape #kaç adet gözlem ve değişken olduğunu görmek için

In [ ]:
df.isna().sum() # daha sonra bu konuya geleceğiz fakat şimdiden eksik gözlem var mı görmek için kullandık

In [ ]:
df.count() #her bir değişkende kaç adet değer olduğunu sorguluyoruz

In [ ]:
df["BataryaGucu"] # ile istediğimiz özellik hakkında sadece o özelliği görüntüleyebiliyoruz.

Çekirdek sayisi ve Ram ilişkisi  Arasında bir korelasyon var.

In [ ]:
df.groupby('CekirdekSayisi')['RAM'].apply(lambda x: np.mean(x))

kolerasyonun kat sayısı

In [ ]:
df.corr()["CekirdekSayisi"]["RAM"]

Görüldüğü üzere ve tahmin edildiği üzere sonuç pozitif ve neredeyse güçlü bir kolerasyon katsayısı


başka bir örnek olarak batarya gücü ve batarya ömrü arasında da bir ilişki kurmak istedim

In [ ]:
df.groupby('BataryaGucu')['BataryaOmru'].apply(lambda x: np.mean(x))

In [ ]:
df.corr()["BataryaGucu"]["BataryaOmru"] # aynı şekilde yine pozitif bir sonuç aldık

# Görselleştirme kısmı


In [ ]:
sns.set(rc={'figure.figsize':(12,10)})  

son yaptığımız örneğin üzerinden devam edebiliriz

In [ ]:
sns.scatterplot(x = "BataryaGucu", y = "BataryaOmru", data = df);

jointplot sayesinde gözlemlerin hangi aralıkta ne kadar yoğun olduğunu da gözlemleyebiliriz.

In [ ]:
sns.jointplot(x = "BataryaGucu", y = "BataryaOmru", data = df, color="yellow");

In [ ]:
df.cov() #kovarsyan matrisi

şiddet hakkında yorum yapmak için kolerasyon matrisi çizebiliriz

In [ ]:
df.corr() 

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);

Koleraesyon matrisini çizip kat sayıları buradan da görebiliriz

In [ ]:
corr = df.corr()
corr

In [ ]:
sns.scatterplot(x = "FiyatAraligi", y = "RAM", hue = "Renk",  data = df);# burada ram ve fiyat aralığı arasındaki ilişkiyi işin içine renkleri de katarak göstermiş olduk

*****

batarya gücünün dağılımını görmek için

In [ ]:
sns.distplot(df["BataryaGucu"], bins=16, color="purple");

burada arka kamera mp hakkında dağılımı görüyoruz

In [ ]:
sns.distplot(df["ArkaKameraMP"], bins=16, color="purple");

Batarya gücünün ve Ram'in yoğunluğunu daha iyi görebilmek için

In [ ]:
sns.jointplot(x = df["BataryaGucu"], y = df["RAM"], kind = "kde", color = "black");

Violinplot (keman grafiği) , kutu grafiğine benzer, ancak verilerin farklı değerlerde olasılık yoğunluğunu da göstermeleri dışında. Bu grafikler, verilerin medyanı için bir işaretleyici ve standart kutu grafiklerinde olduğu gibi çeyrekler arası aralığı belirten bir kutu içerir. 

In [ ]:
sns.violinplot(y = "BataryaGucu", data = df);

In [ ]:
sns.violinplot(y = "RAM", data = df);

Keman grafiğinde iki özellik ile karşılaştırma yapıp ilişki de kurabiliriz.

In [ ]:
sns.violinplot(x = "Renk", y = "Kalinlik", data = df);

Fiyat Aralığı tablet satımında ayırt edici özellik midir ?

In [ ]:
sns.pairplot(df, hue = "FiyatAraligi", palette="Set2");

Çözünürlük yüksekliği ve fiyat aralığı arasındaki ilişki

In [ ]:
sns.barplot(x ="CozunurlukYükseklik" , y = "FiyatAraligi" , data = df);

In [ ]:
sns.lineplot("FiyatAraligi" , "MikroislemciHizi", "CekirdekSayisi", color="black", data=df);

In [ ]:
sns.barplot(x ="BataryaGucu" , y = "FiyatAraligi" , data = df);

In [ ]:
sns.countplot(x = "FiyatAraligi", data = df);# fiyat aralığı dağılımına göz atalım

# Veri Ön İşleme - Hazırlama

In [ ]:
df = pd.read_csv('../input/tabletpc-priceclassification/tablet.csv')

Veri Setini Tanımak ve Sindirmek

In [ ]:
df.info()

In [ ]:
df.shape

Değişkenlerin tipleri nedir ?

In [ ]:
df.dtypes

In [ ]:
df.describe().T

### Benzersiz Değerleri Görüntüleme

In [ ]:
df["3G"].unique()

In [ ]:
df["Bluetooth"].unique()

In [ ]:
df["CiftHat"].unique()

In [ ]:
df["4G"].unique()

In [ ]:
df["WiFi"].unique()

In [ ]:
df["FiyatAraligi"].unique()

In [ ]:
df["Renk"].unique()

### Benzersiz Değerlerin Dağılımı


In [ ]:
df["Bluetooth"].value_counts()

Acaba veri çerçevesinde eksik gözlem var mı? Veri çerçevesi hakkında daha detaylı bilgiler edinelim.

Hangi değişkende kaç adet eksik gözlem var?

In [ ]:
df.isnull().sum()

In [ ]:
df["FiyatAraligi"].value_counts()

## Eksik Değerler 

Bu kısımda eksik veri var mı ona bakacağız , var ise bunu en az hatayla kurtarabilmek için nasıl bir yol izleyebiliriz onu göreceğiz

Eksik Değerlerin Tespiti 

In [ ]:
df.isnull().sum().sum() # toplam 17 tane eksik değerimiz vardır

In [ ]:
df.isnull().sum() #kategorize edilmiş halde eksik değerlerimizi daha iyi görebiliriz

Eksik Verili Değişkenlerin İlişkisi

In [ ]:
missingno.matrix(df,figsize=(20, 10));

In [ ]:
missingno.heatmap(df, figsize= (15,8));

Bu eksik gözlemler toplam gözlem sayısının yüzde kaçına tekabül ediyor?

In [ ]:
def eksik_deger_tablosu(df): 
    eksik_deger = df.isnull().sum()
    eksik_deger_yuzde = 100 * df.isnull().sum()/len(df)
    eksik_deger_tablo = pd.concat([eksik_deger, eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return eksik_deger_tablo_son

In [ ]:
eksik_deger_tablosu(df)

Sıra geldi eksik verileri doldurma kısmına


Burada hata payının en az olması için ve eksik değer sayısının az olması nedeniyle eksik değerler yerine o veri grubunun medyanını yazalım ama ilk önce unique() komutu ile onlara göz atalım

In [ ]:
df["OnKameraMP"].unique()

daha detaylı bakmak için 

In [ ]:
df[df["OnKameraMP"].isnull()]

eksik bilginin olduğu tabletler hakkında bütün bilgilere ulaşabiliriz.

In [ ]:
df["RAM"].unique()

In [ ]:
df[df["RAM"].isnull()]

In [ ]:
df["OnKameraMP"]=df["OnKameraMP"].fillna(df["OnKameraMP"].median())#fillna komutu ile yeni değişkenimi eksiklerin yerine atadık

In [ ]:
df["RAM"]=df["RAM"].fillna(df["RAM"].median())

In [ ]:
df["OnKameraMP"].unique()# görüldüğü üzere eksik değerimiz kayboldu

In [ ]:
df["RAM"].unique()

## Kategorik Değerleri Dönüştürme 

Label / One-Hot Encoding 

Çift hat , 4g gibi birkaç değişkenimizde bulunan "var" ve "yok" ifadeleri bilgisayarın anlayacağı formatta değildir. Dolayısıyla label encoding yapmak için çağırdığımız kütüphaneden bir değişken oluşturalım ve yok, var değerleri için 0, 1 dönüşümü yapalım.

biraz ters düşebilir ama bizim dönüşümümüzde yok = 1       var = 0 oldu :)

In [ ]:
label_encoder = preprocessing.LabelEncoder()# burada dönüşüm için label_encoder kullanacağız

hepsini ayrı ayrı dönüştürüyoruz

In [ ]:
df['Bluetooth'] = label_encoder.fit_transform(df['Bluetooth'])
df.head()


In [ ]:
df['CiftHat'] = label_encoder.fit_transform(df['CiftHat'])
df.head()


In [ ]:
df['4G'] = label_encoder.fit_transform(df['4G'])
df.head()


In [ ]:
df['3G'] = label_encoder.fit_transform(df['3G'])
df.head()


In [ ]:
df['Dokunmatik'] = label_encoder.fit_transform(df['Dokunmatik'])
df.head()


In [ ]:
df['WiFi'] = label_encoder.fit_transform(df['WiFi'])
df.head()


In [ ]:
df['FiyatAraligi'] = label_encoder.fit_transform(df['FiyatAraligi'])
df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
x= [['Beyaz', 0], ['Pembe', 1], ['Mor', 2],['Turuncu',3],['Gri',4],['Sarı',5],['Mavi',6],
   ['Turkuaz',7],['Kahverengi',8],['Yeşil',9],['Kırmızı',10],['Siyah',11]]
df["Renk"]=encoder.fit_transform(df[["Renk"]]).toarray()

In [ ]:
df.head()# görüldüğü üzere veri setimizin değişkenlerinin aldığı değerler istediğimiz hale dönüştü

Böylece bütün veri tiplerimiz makine diline uygun hale getirilmiş oldu.


Ayrıca FiyatAraligi değişkenimiz sıralanabilir sayılaştırma için uygundur

In [ ]:
###categories=pd.Categorical(df["FiyatAraligi"],categories=["Cok Ucuz","Ucuz","Normal","Pahalı"],ordered=True)categories

### hatayı bulamadım o yüzden kaydetmeden devam ediyorum

In [ ]:
df.head(4)

 # makine öğrenmesi modellemeleri

In [ ]:
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB

Bağımlı ve bağımsız değişkenlerimizi atıyoruz


Bağımlı değişkenimiz FiyatAraligi


bağımsız değişkenimiz ise FiyatAraligi dışındaki diğer veriler

In [ ]:
y = df['FiyatAraligi']
X = df.drop(['FiyatAraligi'], axis=1)

Bağımlı ve bağımsız değişkenler üzerinden eğitim ve test olmak üzere ayırım yapıyoruz. Test oranını 25% belirlememiz istenmiş.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

Önce oluşturduğumuz eğitim ve testlere göz atalım

In [ ]:
y

In [ ]:
X

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

Şimdi GaussianNB modeli yaratalım ve eğitimi gerçekleştirelim.

In [ ]:
nb = GaussianNB() # modeli inşa ediyoruz.
nb_model = nb.fit(X_train, y_train) 

In [ ]:
nb_predict=nb_model.predict(X_test)

doğruluk skorumuza bakalım

In [ ]:
nb_model.score(X_test,y_test)

F1_Score, Precision, Recall değerleri içeren “classification_report”umuz

In [ ]:
print(classification_report(y_test,nb_predict))

Karmaşıklık matrisimizi yazdıralım

In [ ]:
confusion_matrix(y_test,nb_predict)

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, f1_score, precision_score
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.metrics import confusion_matrix as cm


filterwarnings('ignore')

bu sefer karar ağaçları için modelimizi inşa edelim ve eğitelim.

In [ ]:
#cart = DecisionTreeClassifier(random_state = 42)
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy') # öneri sonucu entropy parametresi atanmıştır
cart_model = cart.fit(X_train, y_train)

Modelin parametrelerini ve aldığı varsayılan değerleri görüntülemek için model ismini yazabiliriz.

In [ ]:
cart_model


In [ ]:
df.columns

In [ ]:
y_pred = cart_model.predict(X_test)

Karar ağacı için görselleştirme kısmına geçebiliriz şimdi doğruluk skorunu görütüleyelim

In [ ]:
accuracy_score(y_test, y_pred)

karmaşıklık matrisini yazdıralım

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Modelin farklı gözlemler ile skorunu tekrar hesaplayalım

In [ ]:
cross_val_score(cart_model, X, y, cv = 10)

10 skorun ortalaması:

In [ ]:
cross_val_score(cart_model, X, y, cv = 10).mean()

Hedef özniteliğin değerlerine göre sınıflandırma raporunu yazdıralım.

In [ ]:
print(classification_report(y_test, y_pred))

**********

 KNN modeli kuruyoruz ve eğitiyoruz.

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

In [ ]:
knn_params = {"n_neighbors": np.arange(2,15)}#2 ile 15 arası sayıları deneyerek skoru oluşturmaktadır

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(10)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test)

Tüm gözlemler için bulunduğu tahminlere göre doğruluk skorunu görüntüleyelim.

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
score_list = []

for each in range(2,15):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    score_list.append(knn2.score(X_test, y_test))
    
plt.plot(range(2,15),score_list)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")
plt.show()

In [ ]:
knn_model

In [ ]:
y_pred = knn_model.predict(X)

doğruluk skorunu görüntüleyelim.

In [ ]:
accuracy_score(y, y_pred)

Karmaşıklık matrisini yazdılarım

In [ ]:
karmasiklik_matrisi = confusion_matrix(y, y_pred)
print(karmasiklik_matrisi)

In [ ]:
print(classification_report(y, y_pred))

Doğruluk skorları sıralaması KNN > Karar Agacı > GaussianNB şeklindedir. En başarılı sonuca KNN modeli ile ulaşılmıştır.

Ayırdığınız zaman için teşekkür ederim